# Trim conditions for pyMAV

1. 수치적으로 트림점을 찾는 함수 작성
2. 사용자가 지정할 트림조건을 목록화
3. 트림점을 찾을 범위 지정

상태변수와 제어명령

$x \triangleq (p_n, p_e, p_d, u, v, w, \phi, \theta, \psi, p, q, r)^T$ 

$ u \triangleq (\delta_e, \delta_t, \delta_a, \delta_r)^T$

트림점
- Aerodynamic literature에서, 평형점에 있는 항공기를 트림상태에 있다고 부른다.
- 일반적으로 트림조건에서는 일정한 값이 아닌 상태변수도 포함할 수 있다.
- 트림조건에서는 바람이 0인 것으로 가정하고, 바람은 외란으로 고려한다.
    - 따라서, $V_a = V_g, \psi = \chi, \gamma = \gamma_a$ 이다.

트림조건
1. 항공기가 일정한 속력으로 순항한다. $V_a^*$.
2. 항공기가 일정한 비행경로각으로 상승한다. $\gamma^*$.
3. 항공기가 일정한 반지름으로 궤도운동한다. $R$.

트림조건 2.
- $p_n, p_e$는 아무래도 상관 없다.
- $\dot{u} = \dot{v} = \dot{w} = 0$
- $\dot{\phi} = \dot{\theta} = \dot{p} = \dot{q} = \dot{r} = 0$
- $\dot{\psi} = \frac{V_a^*}{R^*}\cos\gamma^*$
- $\dot{h} = V_a^* \sin\gamma^*$
- 결국 $V_a^*, \gamma^*, R^*$ 이 주어지는 경우 모든 상태변수를 지정할 수 있다.


트림점 찾는 순서

1. $V_a^*, \gamma^*, R^*$가 주어진다.
2. $\alpha^*, \beta^*, \phi^*$를 찾는다.
    > $(\alpha^*, \phi^*, \beta^*) = \argmin \|\dot{x}^* - f(x^*, u^*)\|^2$
3. 트림상태변수 $x^*$를 찾는다.
4. 트림제어명령 $u^*$를 찾는다.

In [27]:
import numpy as np
from math import cos, sin, tan, pi, sqrt
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [28]:
# Trim condition
tc = dotdict() # trim condition
tc.Va = 200
tc.gamma = 0
tc.R = 1000
def get_trim_vector(trim_condition):
    return [trim_condition.Va, trim_condition.gamma, trim_condition.R]

In [44]:
# Compute states from (Va, gamma, R), (alpha, beta, phi)
def compute_uvwpqr(Va, gamma, R, alpha, beta, phi):
    u = Va*cos(alpha)*cos(beta)
    v = Va*sin(beta)
    w = Va*sin(alpha)*cos(beta)
    theta = alpha + gamma
    p = -Va/R*sin(theta)
    q = Va/R*sin(phi)*cos(theta)
    r = Va/R*cos(phi)*cos(theta)
    uvw = [u, v, w]
    pqr = [p, q, r]
    return uvw, pqr, theta

alpha = pi/10
beta = 0
phi = 0

uvw, pqr, theta = compute_uvwpqr(*get_trim_vector(tc), alpha, beta, phi)
print(uvw)
print(pqr)
print(theta)


[190.2113032590307, 0.0, 61.803398874989476]
[-0.06180339887498948, 0.0, 0.1902113032590307]
0.3141592653589793


In [71]:
# Compute trim control
def compute_delta_e(inertia_matrix, aircraft_param, rho, aero_coef, Vab, pqr):
    Jxz = inertia_matrix['Jxz']
    Jx = inertia_matrix['Jx']
    Jz = inertia_matrix['Jz']
    c = aircraft_param['c']
    S = aircraft_param['S']
    Cm0 = aero_coef['Cm0']
    Cma = aero_coef['Cma']
    Cmq = aero_coef['Cmq']
    Cmde = aero_coef['Cmde']
    Va, alpha, beta = np.reshape(Vab, (3))
    p, q, r = np.reshape(pqr, (3))
    delta_e = ((Jxz*(p**2-r**2)+(Jx-Jz)*p*r)/(0.5*rho*Va**2*c*S) - Cm0 - Cma*alpha - 0.5*Cmq*c*q/Va)/Cmde
    return delta_e

def compute_delta_ar(Gamma_i, aircraft_param, rho, aero_coef, Vab, pqr):
    G1, G2, G3, G4, G5, G6, G7, G8 = Gamma_i
    S = aircraft_param['S']
    b = aircraft_param['b']
    Cp0 = aero_coef['Cp0']
    Cpb = aero_coef['Cpb']
    Cpp = aero_coef['Cpp']
    Cpr = aero_coef['Cpr']
    Cpda = aero_coef['Cpda']
    Cpdr = aero_coef['Cpdr']
    Cr0 = aero_coef['Cr0']
    Crb = aero_coef['Crb']
    Crp = aero_coef['Crp']
    Crr = aero_coef['Crr']
    Crda = aero_coef['Crda']
    Crdr = aero_coef['Crdr']
    Cr0 = aero_coef['Cr0']
    Va, alpha, beta = np.reshape(Vab, (3))
    p, q, r = np.reshape(pqr, (3))
    mtx1 = np.linalg.inv([[Cpda, Cpdr], [Crda, Crdr]])
    mtx2 = [[(-G1*p*q + G2*q*r)/(0.5*rho*Va**2*S*b) - Cp0 - Cpb*beta - 0.5*Cpp*b*p/Va - 0.5*Cpr*b*r/Va],
            [(-G7*p*q + G1*q*r)/(0.5*rho*Va**2*S*b) - Cr0 - Crb*beta - 0.5*Crp*b*p/Va - 0.5*Crr*b*r/Va]]
    delta_a, delta_r = np.reshape(np.matmul(mtx1, mtx2), (2))
    return delta_a, delta_r

print(pqr)
inertia_matrix = dotdict()
inertia_matrix.Jxz = 1
inertia_matrix.Jx = 100
inertia_matrix.Jz = 100
Gamma_i = [0,0,0,0,0,0,0,0]
aircraft_param = dotdict()
aircraft_param.S = 100
aircraft_param.c = 20
aircraft_param.b = 20
rho = 1
aero_coef = dotdict()
aero_coef.Cm0 = 1
aero_coef.Cma = 2
aero_coef.Cmq = 1
aero_coef.Cmde = 0.1
aero_coef.Cp0 = 1
aero_coef.Cpb = 1
aero_coef.Cpp = 1
aero_coef.Cpr = 1
aero_coef.Cpda = 3
aero_coef.Cpdr= 4
aero_coef.Cr0 = 1
aero_coef.Crb = 1
aero_coef.Crp = 1
aero_coef.Crr = 1
aero_coef.Crda = 1
aero_coef.Crdr = 2
Vab = [50, pi/10, 0]
delta_e = compute_delta_e(inertia_matrix, aircraft_param, rho, aero_coef, Vab, pqr)
delta_a, delta_r = compute_delta_ar(Gamma_i, aircraft_param, rho, aero_coef, Vab, pqr)
print(delta_e, delta_a, delta_r)

def compute_delta_t(aircraft_param, aero_coef, Vab, uvw, pqr, theta):
    S = aircraft_param['S']
    c = aircraft_param['c']
    g = aircraft_param['g']
    MASS = aircraft_param['MASS']
    S_prop = aircraft_param['S_prop']
    C_prop = aircraft_param['C_prop']
    k_motor = aircraft_param['k_motor']
    CL0 = aero_coef['CL0']
    CLa = aero_coef['CLa']
    CLq = aero_coef['CLq']
    CLde = aero_coef['CLde']
    CD0 = aero_coef['CD0']
    CDa = aero_coef['CDa']
    CDq = aero_coef['CDq']
    CDde = aero_coef['CDde']
    Va, alpha, beta = np.reshape(Vab, (3))
    p, q, r = np.reshape(pqr, (3))
    u, v, w = np.reshape(uvw, (3))
    CX0, CXa, CXq, CXde = [-i*cos(alpha) + j*sin(alpha) for i,j in zip([CL0, CLa, CLq, CLde], [CD0, CDa, CDq, CDde])]
    CX = CX0 + CXa*alpha
    term1 = (2*MASS*(-r*v + q*w + g*sin(theta)) - rho*Va**2*S*(CX + 0.5*CXq*c*q/Va + CXde*delta_e))/(rho*S_prop*C_prop*k_motor**2)
    term2 = Va**2 / k_motor**2
    if term1 + term2 < 0:
        raise ValueError("Invalid parameters.. cannot compute delta_t")
    return sqrt(term1 + term2)

aircraft_param.g = 9.8
aircraft_param.MASS = 1000
aircraft_param.S_prop = 10
aircraft_param.C_prop = 10
aircraft_param.k_motor = 2
aero_coef.CL0 = 1
aero_coef.CLa = 2
aero_coef.CLq = 3
aero_coef.CLde = 2
aero_coef.CD0 = 2
aero_coef.CDa = 4
aero_coef.CDq = 1
aero_coef.CDde = 1
delta_t = compute_delta_t(aircraft_param, aero_coef, Vab, uvw, pqr, theta)

[-0.06180339887498948, 0.0, 0.1902113032590307]
-16.283185436622304 1.0256815808768078 -1.025681580876808


ValueError: Invalid parameters.. cannot compute delta_t